In [93]:
import sys
sys.path.append("..")
from data_prep.prep import MeterDataSet
import pandas as pd
import numpy as np
from webapp.utils.azure_utils import KeyVault, DataLake

In [94]:
# Connect to Storage Account
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)
file_system = "energyhub"

In [95]:
meter = "electricity"
metadata_cols = ['building_id', 'site_id','sq_meter']
weather_cols = ['site_id', 'timestamp', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precipitation_depth_1_hr',
    'precipitation_depth_6_hr', 'sea_level_pressure', 'wind_direction',
    'wind_speed']

In [96]:
bad_building_dir = "/bad_buildings"
data = MeterDataSet(meter, metadata_cols, weather_cols)
bad_buildings = storage.pandas_read(
            file_system, directory=bad_building_dir, file_name="bad_buildings.csv")

In [97]:
# Treat nan for weather:
w_cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precipitation_depth_1_hr',
    'precipitation_depth_6_hr', 'sea_level_pressure', 'wind_direction',
    'wind_speed']
data.weather = data.fill_weather_na(w_cols = w_cols, fill_method = "linear")

In [98]:
# Treat nan for meter:
meter_cols = [meter]
data.meter = data.fill_meter_na(meter_col = meter_cols, fill_method = "linear")

In [99]:
data.weather = data.weather.compute()

In [100]:
data.meter = data.meter.compute()

In [101]:
data.metadata = data.metadata.compute()

In [102]:
df = data.meter.merge(data.metadata, how="left", left_on="building_id", right_on="building_id")
df = df[~df.building_id.isin(bad_buildings["building_id"])]

In [109]:
buildings = df.building_id.unique()

In [136]:
dfs = []

for building in buildings:
    d = df.loc[df.building_id == building, :]
    d = d.set_index("timestamp")
    site_id = d["site_id"][0]
    w = data.weather.loc[data.weather.site_id == site_id, :].set_index("timestamp")
    dw = d.merge(w).reset_index(drop = True)
    dfs.append(dw)

MemoryError: Unable to allocate 617. MiB for an array with shape (9, 8988425) and data type float64